In [6]:
import time
import sqlite3
import pandas as pd
from futu import *


2020-02-25 21:25:13,203 [open_context_base.py] _socket_reconnect_and_wait_ready:256: Start connecting: host=127.0.0.1; port=11111;
2020-02-25 21:25:13,206 [open_context_base.py] on_connected:344: Connected : conn_id=4; 
2020-02-25 21:25:13,209 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=4; info={'server_version': 209, 'login_user_id': 294252, 'conn_id': 6638061174471030082, 'conn_key': '494A4BA4F08FC8D1', 'conn_iv': 'A7BE682101423F4A', 'keep_alive_interval': 10, 'is_encrypt': False};


In [151]:
ctx = OpenQuoteContext(host='127.0.0.1', port=11111)
code_list = ['SH.600200']
print(ctx.get_owner_plate(code_list))
# quote_ctx.close()

2020-02-26 00:05:42,092 [open_context_base.py] _socket_reconnect_and_wait_ready:256: Start connecting: host=127.0.0.1; port=11111;
2020-02-26 00:05:42,096 [open_context_base.py] on_connected:344: Connected : conn_id=9; 
2020-02-26 00:05:42,101 [open_context_base.py] _handle_init_connect:445: InitConnect ok: conn_id=9; info={'server_version': 209, 'login_user_id': 294252, 'conn_id': 6638101560963846280, 'conn_key': '6DCFAFDDF08F8C3C', 'conn_iv': '294E02302647EF4A', 'keep_alive_interval': 10, 'is_encrypt': False};


(0,          code plate_code plate_name plate_type
0   SH.600200  SH.BK0042       化学制药   INDUSTRY
1   SH.600200  SH.BK0115       参股金融      OTHER
2   SH.600200  SH.BK0125       定向增发      OTHER
3   SH.600200  SH.BK0142       股权激励      OTHER
4   SH.600200  SH.BK0158      IPO受益      OTHER
5   SH.600200  SH.BK0163       基金新增      OTHER
6   SH.600200  SH.BK0211      上证380      OTHER
7   SH.600200  SH.BK0218        特种药      OTHER
8   SH.600200  SH.BK0258        再融资      OTHER
9   SH.600200  SH.BK0263      中证500      OTHER
10  SH.600200  SH.BK0338       融资融券    CONCEPT
11  SH.600200  SH.BK0384         江苏    CONCEPT
12  SH.600200  SH.BK0425       股份回购    CONCEPT
13  SH.600200  SH.BK0456       分散染料    CONCEPT
14  SH.600200  SH.BK0516         江苏     REGION
15  SH.600200  SH.BK0543       股权转让    CONCEPT
16  SH.600200  SH.BK0621      转融券标的    CONCEPT
17  SH.600200  SH.BK0629       证金持股    CONCEPT
18  SH.600200  SH.BK0987     沪股通可融资      OTHER)


In [8]:
plate_list_df = ctx.get_plate_list(Market.SH, Plate.CONCEPT)[1]
# plate_list_df.head()

In [9]:
def update_concept_db(dpath):
    concept_list_df = ctx.get_plate_list(Market.SH, Plate.CONCEPT)[1]
    
    concept_dashboard_code = pd.DataFrame()
    concept_dashboard_name = pd.DataFrame()
    counter = 0
    for idx, row in concept_list_df.iterrows():
        print(str(idx) + '  ' + str(row['plate_name']))
        plate_df = ctx.get_plate_stock(row['code'])[1] # 30 秒内请求最多 10 次
        
        concept_dashboard_code = pd.concat([concept_dashboard_code, plate_df['code']], axis=1)
        concept_dashboard_name = pd.concat([concept_dashboard_name, plate_df['stock_name']], axis=1)
        concept_dashboard_code.rename(columns={'code':row['plate_name']}, inplace=True)
        concept_dashboard_name.rename(columns={'stock_name':row['plate_name']}, inplace=True)
        
        counter = counter+1
        if (counter > 9):
            counter = 0
            time.sleep(30)
    
    sql_ctx = sqlite3.connect(dpath)
    concept_list_df.to_sql('concept_list', sql_ctx, if_exists='replace')
    concept_dashboard_code.to_sql('concept_dashboard_code', sql_ctx, if_exists='replace')
    concept_dashboard_name.to_sql('concept_dashboard_name', sql_ctx, if_exists='replace')

In [10]:
ctx.close()

2020-02-25 21:25:17,002 [network_manager.py] work:366: Close: conn_id=5


In [93]:
sql_cmd = "select * from concept_dashboard_name"
context = sqlite3.connect('/Users/linhua/PycharmProjects/Fupan/database/futu.db')
futu_df = pd.read_sql(sql_cmd, context, index_col='index')

In [94]:
df.head(2)

,中韩自贸区,汽车电子概念,高送转,上海自贸区,电力物联网,养鸡,医疗器械,燃料电池,乳业,黄金,...,光刻机,上证50,深圳特区,分拆上市预期,数字货币,VPN,医疗美容,农药兽药,中药,双黄连概念
index,,,,,,,,,,,,,,,,,,,,,
0,青岛双星,富奥股份,泰禾集团,悦心健康,许继电气,康达尔,海王生物,华锦股份,新希望,中国宝安,...,智光电气,浦发银行,平安银行,中国宝安,中国长城,*ST高升,苏宁环球,长春高新,宜华健康,葵花药业
1,冰轮环境,深圳华强,华兰生物,怡亚通,东方电子,新希望,英特集团,潍柴动力,皇氏集团,中金岭南,...,永太科技,民生银行,万科A,深康佳A,智度股份,南洋股份,景峰医药,泰合健康,ST地矿,太龙药业


In [95]:
def lookup_concept(stock_name, plate_dashboard_df):
    # todo new func, not test
    concept_dict = {}
    index = 1
    for label, content in plate_dashboard_df.iteritems():
        if stock_name in content.values:
            concept_dict.update({'concept'+str(index): [label]})
            index += 1
    return concept_dict

In [96]:
src_df = pd.read_csv('/Users/linhua/OneDrive/Stock/tdx/fupan/data/export-当日涨停20200203.txt', sep='\t', encoding="gbk")

In [97]:
src_df.drop(src_df.index[-1], axis=0, inplace=True)
src_df.drop([src_df.columns[-1]], axis=1, inplace=True)
src_df.tail(3)

,代码,名称,涨幅%,现价,涨跌,买价,卖价,总量,现量,涨速%,...,最高,最低,昨收,AB股总市值,市盈(动),总金额,流通市值,量比,细分行业,地区
73,300676,华大基因,1.41,80.10,1.11,80.10,80.11,85659.0,747.0,-0.04,...,81.99,79.90,78.99,320.48亿,89.00,691817920.0,195.85亿,0.87,医疗保健,深圳
74,300677,英科医疗,4.39,34.92,1.47,34.91,34.92,56383.0,1033.0,0.09,...,35.74,32.86,33.45,69.25亿,41.29,193800176.0,39.58亿,0.91,医疗保健,山东
75,300705,九典制药,0.23,13.15,0.03,13.14,13.15,47225.0,465.0,0.15,...,13.27,13.01,13.12,30.86亿,61.14,62093796.0,13.03亿,0.75,化学制药,湖南


In [135]:
stock_list_df = src_df[['名称']]

stock_concept_dashboard = pd.DataFrame()
# print(stock_concept_dashboard)

for idx, row in stock_list_df.iterrows():
    concept_dict = lookup_concept(row['名称'], futu_df)
    stock_concept_row_df = pd.DataFrame.from_dict(concept_dict)
    stock_concept_row_df.insert(0, '名称', row['名称'])
    stock_concept_dashboard = stock_concept_dashboard.append(stock_concept_row_df, ignore_index=True, sort=False)

stock_concept_dashboard.head(3)

,名称,concept1,concept2,concept3,concept4,concept5,concept6,concept7,concept8,concept9,concept10,concept11,concept12,concept13,concept14,concept15
0,中国医药,融资融券,国企改革,中字头,沪股通,参股券商,生物医药,北京,转融券标的,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,江苏吴中,融资融券,江苏,股份回购,分散染料,股权转让,转融券标的,证金持股,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,太龙药业,融资融券,河南,生物医药,禽流感,转融券标的,中药,双黄连概念,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
# algo base verify
# d1 = stock_concept_dashboard.groupby('concept1')['名称'].count().to_frame(name='count')
# d2 = stock_concept_dashboard.groupby('concept2')['名称'].count().to_frame(name='count')
# d1.add(d2)

sum_df = pd.DataFrame()
for label, content in stock_concept_dashboard.iteritems():
    print(label, sep='\n')
    if label == '名称':
        continue
    else:
        group_col_df = stock_concept_dashboard.groupby(label)['名称'].count().to_frame(name='count')
        sum_df = sum_df.add(group_col_df, fill_value=0)
# sum_df.reset_index()

sum_df.sort_values(['count'], ascending=False, inplace=True)
sum_df.reset_index(inplace=True)
sum_df.rename(columns={'count': '0225', 'index': 'Futu概念'}, inplace=True)

sum_df

名称
concept1
concept2
concept3
concept4
concept5
concept6
concept7
concept8
concept9
concept10
concept11
concept12
concept13
concept14
concept15


,Futu概念,0225
0,融资融券,29.0
1,生物医药,21.0
2,医疗器械,19.0
3,转融券标的,18.0
4,中药,16.0
5,深股通,14.0
6,禽流感,11.0
7,超级真菌,10.0
8,生物疫苗,10.0
9,广东,10.0


In [219]:

col_cnt = pd.Series(sum_df.columns.array[0])
col_cnt = col_cnt.append(sum_df['Futu概念'], ignore_index=True)
col_cnt

0          Futu概念
1            融资融券
2            生物医药
3            医疗器械
4           转融券标的
5              中药
6             深股通
7             禽流感
8            超级真菌
9            生物疫苗
10             广东
11             江苏
12             山东
13             上海
14             浙江
15           健康中国
16           工业大麻
17             创投
18         创业板壳资源
19           互联网+
20           股权转让
21           深圳特区
22           单抗概念
23           基因测序
24            沪股通
25           养老概念
26           振兴东北
27           股份回购
28          双黄连概念
29           医药电商
          ...    
113         互联网金融
114           中字头
115         PPP模式
116         O2O概念
117        MSCI概念
118    Facebook概念
119           AH股
120            四川
121           大数据
122           物联网
123           无人机
124           煤化工
125            湖南
126            河南
127            河北
128           氢能源
129           柔性屏
130         机器人概念
131          无线充电
132          5G概念
133        宁德时代概念
134          数字孪生
135          数字中国
136            广西
137       

In [193]:
stock_concept_dashboard.to_csv('verify_table.csv', encoding="gbk")

In [198]:
# to get colors
import seaborn as sns

y = sns.color_palette('Paired', 400).as_hex()

In [220]:
# to get plate size
futu_df

,中韩自贸区,汽车电子概念,高送转,上海自贸区,电力物联网,养鸡,医疗器械,燃料电池,乳业,黄金,...,光刻机,上证50,深圳特区,分拆上市预期,数字货币,VPN,医疗美容,农药兽药,中药,双黄连概念
index,,,,,,,,,,,,,,,,,,,,,
0,青岛双星,富奥股份,泰禾集团,悦心健康,许继电气,康达尔,海王生物,华锦股份,新希望,中国宝安,...,智光电气,浦发银行,平安银行,中国宝安,中国长城,*ST高升,苏宁环球,长春高新,宜华健康,葵花药业
1,冰轮环境,深圳华强,华兰生物,怡亚通,东方电子,新希望,英特集团,潍柴动力,皇氏集团,中金岭南,...,永太科技,民生银行,万科A,深康佳A,智度股份,南洋股份,景峰医药,泰合健康,ST地矿,太龙药业
2,中广核技,风华高科,双鹭药业,海得控制,思源电气,民和股份,海南海药,东旭光电,大康农业,ST地矿,...,胜利精密,宝钢股份,国农科技,铜陵有色,恒宝股份,卫士通,华东医药,海翔药业,东阿阿胶,天士力
3,青岛金王,中鼎股份,中钢天源,*ST飞马,远光软件,圣农发展,京东方A,穗恒运A,洽洽食品,中润资源,...,西陇科学,中国石化,世纪星源,长春高新,广电运通,启明星辰,双鹭药业,天康生物,丽珠集团,哈药股份
4,朗姿股份,航天科技,金智科技,滨江集团,金智科技,华英农业,重药控股,江铃汽车,贝因美,金圆股份,...,光华科技,南方航空,深振业A,盈峰环境,亚联发展,二六三,朗姿股份,金河生物,云南白药,ST辅仁
5,日照港,华工科技,三钢闽光,中远海科,科陆电子,益生股份,欣龙控股,苏常柴A,麦趣尔,金洲慈航,...,宝通科技,中信证券,全新好,华邦健康,众应互联,南兴股份,哈三联,绿康生化,启迪古汉,人民同泰
6,锦州港,宁波华翔,东港股份,摩恩电气,智光电气,仙坛股份,京新药业,*ST大洲,燕塘乳业,盛达资源,...,金龙机电,三一重工,中国宝安,东港股份,金固股份,朗源股份,冠昊生物,瑞普生物,吉林敖东,珍宝岛
7,时代万恒,得润电子,湘潭电化,华测检测,威创股份,华统股份,科华生物,英力特,科迪乳业,银泰黄金,...,上海新阳,招商银行,*ST美丽,西部材料,海联金汇,蓝盾股份,常山药业,东湖高新,仁和药业,None
8,营口港,顺络电子,合肥城建,新宁物流,积成电子,温氏股份,达安基因,茂化实华,庄园牧场,ST冠福,...,蓝英装备,保利地产,深物业A,广电运通,奥马电器,任子行,鲁商发展,中牧股份,通化金马,None


In [227]:
x=futu_df.count().to_dict()
x

{'中韩自贸区': 15,
 '汽车电子概念': 71,
 '高送转': 173,
 '上海自贸区': 68,
 '电力物联网': 38,
 '养鸡': 11,
 '医疗器械': 102,
 '燃料电池': 124,
 '乳业': 22,
 '黄金': 46,
 '智能音箱': 15,
 '微信小程序': 12,
 '新疆': 55,
 '福建': 137,
 '健康中国': 93,
 '雄安新区': 178,
 '边缘计算': 79,
 '太阳能': 77,
 '养老概念': 53,
 '短视频': 21,
 '国产软件': 128,
 '煤化工': 82,
 '天然气': 103,
 '网约车': 19,
 '海工装备': 24,
 '阿里概念': 98,
 '长三角一体化': 69,
 '大数据': 174,
 '互联网+': 207,
 '铁路基建': 38,
 '冷链物流': 30,
 '军民融合': 140,
 '沪伦通': 3,
 'OLED': 104,
 '手游概念': 87,
 '人脸识别': 29,
 '物联网': 186,
 '知识产权保护': 37,
 '融资融券': 1634,
 '农机': 15,
 '河南': 79,
 '贵州': 29,
 '高铁': 55,
 '超级真菌': 17,
 '国企改革': 309,
 '智能医疗': 46,
 '乡村振兴': 82,
 '移动支付': 59,
 '债转股': 51,
 '中字头': 43,
 '啤酒': 9,
 '上海国资改革': 67,
 '智能穿戴': 77,
 '山西': 37,
 '钛白粉': 6,
 '天津自贸区': 13,
 'LED': 70,
 '锂电池概念': 200,
 '垃圾分类': 95,
 '杭州湾大湾区': 10,
 '地热能': 16,
 '蚂蚁金服概念': 43,
 '云计算': 132,
 '区块链': 150,
 '猪肉': 28,
 '白马股': 140,
 '独角兽概念': 187,
 '网络安全': 72,
 '民营医院': 44,
 '数字孪生': 18,
 '数字中国': 41,
 '透明工厂': 27,
 '钴': 29,
 '特色小镇': 30,
 '共享单车': 11,
 '体育产业': 53,
 '宽带中国': 30,
 '迪士尼':

中韩自贸区      15
汽车电子概念     71
高送转       173
上海自贸区      68
电力物联网      38
养鸡         11
医疗器械      102
燃料电池      124
乳业         22
黄金         46
智能音箱       15
微信小程序      12
新疆         55
福建        137
健康中国       93
雄安新区      178
边缘计算       79
太阳能        77
养老概念       53
短视频        21
国产软件      128
煤化工        82
天然气       103
网约车        19
海工装备       24
阿里概念       98
长三角一体化     69
大数据       174
互联网+      207
铁路基建       38
         ... 
高送转填权      47
华为概念      325
海绵城市       48
通用航空       29
氢能源        68
智慧停车       12
北京        334
IPV6       17
虚拟现实      112
转融券标的     948
农村电商       26
深圳国资改革     23
智能电视       26
内蒙古        25
触摸屏概念      33
AB股        79
工业大麻       66
证金持股      150
ETC        35
磷化工        33
光刻机        22
上证50       50
深圳特区      291
分拆上市预期     28
数字货币       23
VPN        18
医疗美容       11
农药兽药       14
中药         87
双黄连概念       7
Length: 296, dtype: int64